In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
 
#To reset seed so same set of numbers are produced
np.random.seed(0)

#inputing the dataset
data = pd.read_csv('LBW_Dataset.csv') 
data.head(5)
data.info()

#Count of null values column wise
data.isnull().sum()

#Inspect unique values in every feature
for col in data.columns:
    print("{}:{}".format(col,data[col].nunique()))

#Basic statistics feautre wise
data.describe()

#NO null values in community column
data['Community'].value_counts()

data['Residence'].value_counts()

# FIll Age ,Weight ,HB and BP  Null values with mean
# Delivery phase with 1 since the coulmn takes binary values
# Education with 5
#Residence with 1

#returns all the features
data.columns

#used to fill the "HB" feature by mean
x=data['HB'].mean()
print(x)

# A dict containing column wise values used to fill the null values
values = { 'Age':data['Age'].mean(), 'Weight':data['Weight'].mean(), 'Delivery phase':1, 'HB':9.0, 'BP':data['BP'].mean(),
       'Education':5, 'Residence':1}
data = data.fillna(value=values)
data.head(3)
data.isnull().sum() #no nulls values the dataset is now cleaned

#Writing the cleaned output to new file
data.to_csv("LBW_Dataset_cleaned.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Community       96 non-null     int64  
 1   Age             89 non-null     float64
 2   Weight          85 non-null     float64
 3   Delivery phase  92 non-null     float64
 4   HB              77 non-null     float64
 5   IFA             96 non-null     int64  
 6   BP              81 non-null     float64
 7   Education       93 non-null     float64
 8   Residence       94 non-null     float64
 9   Result          96 non-null     int64  
dtypes: float64(7), int64(3)
memory usage: 7.6 KB
Community:4
Age:14
Weight:28
Delivery phase:2
HB:22
IFA:2
BP:20
Education:1
Residence:2
Result:2
9.076623376623376


In [2]:
'''
Design of a Neural Network from scratch

*************<IMP>*************
Mention hyperparameters used and describe functionality in detail in this space
- carries 1 mark
'''
class NN:
    
    ''' X and Y are dataframes'''
    def __init__(self,numHlayers=2):
        
        #number of hidden layers the ANN has
        self.numHlayers = numHlayers
    
    def fit(self,X,Y):
        '''
        Function that trains the neural network by talking x_train and y_train samples as input
        '''
        input1=X
        result1=Y
        
        #If the ANN has two hidden layer we initialize it calling ANN_layer class
        
        if(self.numHlayers==2):
            
            #First layer is initialized with parameter 1 :Number of inputs
            #                                parameter 2 :Number of neurons
            #similarly hidden layer and output layer is initialised
            self.Hlayer1 =ANN_layers(7,24)
            self.Hlayer2 =ANN_layers(24,24)
            self.Output_layer =ANN_layers(24,2)
            
            #Between each layer we have the activation function class with a 
            #wide range of activation functions 
            
            self.act1=activation()
            self.act2=activation()
            
            # using the softmax function on the final layer 
            self.loss_activation=Activation_softmax_LCCE()
            
            #So for optimizer we can choose between Adam and SGD optimizer
            #Creating optimizer
            self.optimize=optimizer(l_rate=0.05,decay=5e-7)
            
            
            #Loop to train in
            for epoch in range(10001):
                
                #pass input X through first layer
                self.Hlayer1.fpropogation(input1)
                
                #Output of first layer passes through the firstactivation function
                self.act1.fRelu(self.Hlayer1.output)
                
                #Output of first activation is sent in forward pass to second layer
                self.Hlayer2.fpropogation(self.act1.output)
                
                 #Output ofsecond layer passes through the next activation function
                self.act2.fRelu(self.Hlayer2.output)
                
                # Perform a forward pass through the activation/loss function
                # takes the output of second dense layer here and returns loss
                self.Output_layer.fpropogation(self.act2.output)
                loss = self.loss_activation.forward(self.Output_layer.output,result1)
                
                #Argmax returns the predictions for the foward pass
                prediction = np.argmax(self.loss_activation.output,axis=1)
                
                #calculate values along first axis
                if len(result1.shape)==2:
                    result1=np.argmax(result1,axis=1)
                #returns the accuracy of our predictions in this pass
                accuracy= np.mean(prediction==result1)
                
                #priniting for every 100Th epoch 
                if not epoch %100:
                    print(f'epoch:{epoch} ,'+
                          f'accuracy:{accuracy:.3f} ,'+
                          f'loss:{loss} ,'+
                          f'lr:{self.optimize.curr_l_rate},')
                    
                #back propogation 
                #The values are propogated backwards and the dinputs are updated
                self.loss_activation.backpass(self.loss_activation.output,result1)
                self.Output_layer.Bpropogation(self.loss_activation.dinputs)
                self.act2.back(self.Output_layer.dinputs)
                self.Hlayer2.Bpropogation(self.act2.dinputs)
                self.act1.back(self.Hlayer2.dinputs)
                self.Hlayer1.Bpropogation(self.act1.dinputs)
                
                
                #Update in weights and the biases
                self.optimize.initial_update_param()
                self.optimize.update_params(self.Hlayer1)
                self.optimize.update_params(self.Hlayer2)
                self.optimize.update_params(self.Output_layer)
                self.optimize.post_update_params()
        
        
        #If ANN has 1 hidden layer we initialize it calling ANN_layer()
        
        if(self.numHlayers==1):
            
            # layers are initialized with parameter 1 :Number of inputs
            #                             parameter 2 :Number of neurons
            #similarly hidden layer and output layer is initialised
            
            self.Hlayer1 =ANN_layers(7,64)
            self.Output_layer =ANN_layers(64,2)
            #Between each layer we have the activation function class with a 
            #wide range of activation functions 
            
            self.act1=activation()
            
            # using the softmax function on the final layer
            self.loss_activation=Activation_softmax_LCCE()
            
            #So for optimizer we can choose between Adam and SGD optimizer
            #Creating optimizer
            self.optimize=optimizer(l_rate=0.05,decay=5e-7)
            
            #Loop to train in
            for epoch in range(11001):
                
                
                #pass input X through first layer
                #Output of first layer passes through the firstactivation function
                # Perform a forward pass through the activation/loss function
                # takes the output of second dense layer here and returns loss
                self.Hlayer1.fpropogation(input1)
                self.act1.fRelu(self.Hlayer1.output)
                self.Output_layer.fpropogation(self.act1.output)
                loss = self.loss_activation.forward(self.Output_layer.output,result1)
                
                #Argmax returns the predictions for the foward pass
                prediction = np.argmax(self.loss_activation.output,axis=1)
                
                #calculate values along first axis
                if len(result1.shape)==2:
                    result1=np.argmax(result1,axis=1)
               
                #returns the accuracy of our predictions in this pass
                accuracy= np.mean(prediction==result1)
               
                
                #priniting for every 100Th epoch
                if not epoch %100:
                    print(f'epoch:{epoch} ,'+
                          f'accuracy:{accuracy:.3f} ,'+
                          f'loss:{loss} ,'+
                          f'lr:{self.optimize.curr_l_rate},')
                
                
                #back propogation 
                #The values are propogated backwards and the dinputs are updated
                
                self.loss_activation.backpass(self.loss_activation.output,result1)
                self.Output_layer.Bpropogation(self.loss_activation.dinputs)
                self.act1.back(self.Output_layer.dinputs)
                self.Hlayer1.Bpropogation(self.act1.dinputs)
                
                
                #Update in weights and the biases

                self.optimize.initial_update_param()
                self.optimize.update_params(self.Hlayer1)
                self.optimize.update_params(self.Output_layer)
                self.optimize.post_update_params()
                
               
    
    
    def predict(self,X,Y):

        """
        The predict function performs a simple feed forward of weights
        and outputs yhat values 

        yhat is a list of the predicted value for df X
        
        Only Forward pass to be performed in already trained model
        """
        
        #Only Forward pass to be performed in already trained model
        x_test=X
        y_test=Y
        #If ANN has 2 hidden layer we call the  ANN_layer() objects
        if(self.numHlayers==2):
            #pass input to the first layer 
            self.Hlayer1.fpropogation(x_test)
            
            #layer 1 activation func
            self.act1.fRelu(self.Hlayer1.output)
            
            #output of act1 to 2nd layer
            self.Hlayer2.fpropogation(self.act1.output)
            
            #layer 2 activation func
            self.act2.fRelu(self.Hlayer2.output)
            self.Output_layer.fpropogation(self.act2.output)
            
            #loss/activation for output layer
            loss = self.loss_activation.forward(self.Output_layer.output,y_test)
            
            #Argmax returns our models predictions
            prediction = np.argmax(self.loss_activation.output,axis=1)
            if len(y_test.shape)==2:
                result1=np.argmax(y_test,axis=1)
            
            #Accuracy out model has attained
            accuracy= np.mean(prediction==y_test)
            print(f'accuracy:{accuracy:.3f} ,'+
                  f'lr:{self.optimize.curr_l_rate},')
            
            #return the prediction
            yhat=prediction
            return yhat
       
    
        #If ANN has 1 hidden layer we call the  ANN_layer() objects
        if(self.numHlayers==1):
            #pass input to the first layer 
            self.Hlayer1.fpropogation(x_test)
            
            #layer 1 activation func
            self.act1.fRelu(self.Hlayer1.output)
            
            self.Output_layer.fpropogation(self.act1.output)
            #loss/activation for output layer
            loss = self.loss_activation.forward(self.Output_layer.output,y_test)
            
            #Argmax returns our models predictions
            prediction = np.argmax(self.loss_activation.output,axis=1)
            if len(y_test.shape)==2:
                result1=np.argmax(y_test,axis=1)
            
            #Accuracy out model has attained
            accuracy= np.mean(prediction==y_test)
            print(f'accuracy:{accuracy:.3f} ,'+
                  f'lr:{self.optimize.curr_l_rate},')
            #return the prediction
            yhat=prediction
            return yhat
        
    
    def CM(self,y_test,y_test_obs):
        
        """
        Since our prediction are already given in list np.array we dont 
        need to convert confidence values
        """
        #2x2 matrix 
        cm=np.array([[0,0],[0,0]])
        fp=0 #false positive
        fn=0 #false negative
        tp=0 #true Positive
        tn=0 #true negative
        
        #loop over number of samples
        for i in range(len(y_test)):
            if(y_test[i]==1 and y_test_obs[i]==1):
                tp=tp+1
            if(y_test[i]==0 and y_test_obs[i]==0):
                tn=tn+1
            if(y_test[i]==1 and y_test_obs[i]==0):
                fp=fp+1
            if(y_test[i]==0 and y_test_obs[i]==1):
                fn=fn+1
        cm[0][0]=tn
        cm[0][1]=fp
        cm[1][0]=fn
        cm[1][1]=tp
        
        p= tp/(tp+fp) #precision
        r=tp/(tp+fn)  #recall
        f1=(2*p*r)/(p+r) #f1 score
        
        
        print("Confusion Matrix : ")
        print(cm)
        print("\n")
        print(f"Precision : {p}")
        print(f"Recall : {r}")
        print(f"F1 SCORE : {f1}")
        
        
                
                
            

In [3]:
#layer class so each layer can be intilialised as an object 
class ANN_layers:
    
    # layer properties
    def __init__(self,num_inputs ,num_cells):
        
        #initializing weights and biases
        self.weights = 0.1 * np.random.randn(num_inputs,num_cells) #in range[-1,1]
        self.biases = np.zeros((1,num_cells))
        #self.w_cahce=np.zeros_like(self.weights)
    
    #forward pass
    def fpropogation(self , inputs):
        
        #make copy of inputs
        self.inputs= inputs
        
        #Calculate output values from inputs,weights and biases
        self.output= np.dot(inputs ,self.weights)
    #back pass
    def Bpropogation(self,dval):
        #gradient on parameters and values
        self.dweights= np.dot(self.inputs.T,dval)
        self.dbiases = np.sum(dval,axis=0,keepdims=True)
        self.dinputs =np.dot(dval,self.weights.T)

def confidenceTolist(x):
    #to convert a given confidence matrix to list of predictions
    rows = x.shape[0]
    cols = x.shape[1]
    predict=[None]*rows
    for i in range(0,rows):
        if(x[i,0]>x[i,1]):
            predict[i]=0
        else:
            predict[i]=1
    return predict
        
        
    
#Activation class with different activation func        
class activation:
    
    #RELU activation
    #makes copy of input values
    def fRelu(self,x):
        self.inputs=x
        #calculates and stores the output
        self.output = np.maximum(0,x)
        #sigmoid
        #self.inputs =x
        #self.output = 1/(1+np.exp(-x))
        #self.output = np.tanh(x) #tanh
   
    #sigmoid activation
    def sigmoid(self,x):
        self.inputs =x
        self.output = 1/(1+np.exp(-x))
    
    #tanh activation
    def tanh(self,x):
        self.inputs =x
        self.output = np.tanh(x)
    
    #for back pass    
    def back(self,dval):
        #copy of back pass inputs
        self.dinputs = dval.copy()
        
        #Gradient to be set ) for -ve values
        self.dinputs[self.inputs <=0] =0 
        
#Softmax activation
class act_softmax :       
    
    #forward pass
    def fsoftmax(self,inputs):
        #make copy of input values
        self.inputs = inputs
        
        #unnormalised probabities
        exp1 = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        #normalising 
        prob = exp1 /np.sum(exp1 ,axis=1, keepdims=True)
        
        #storing and calculating output
        self.output = prob
    
    #Back pass
    def bsoftmax(self,dval):
        
        #create empty array
        self.dinputs = np.empty_like(dval)
        
        #enumerate output and gradients
        for i ,(single_o,single_dval) in enumerate(zip(self.output,dval)):
            
            #reshape output array
            single_o = single_o.reshape(-1,1)
            
            #Calculate the jacobian matrix of the output
            jacobian = np.diagflat(single_o)-np.dot(single_o,single_o.T)
            
            
            self.dinputs[i] = np.dot(jacobian,single_dval)

#Loss class             
class loss:
    
    #for given model output and ground truth values calculate loss
    def cal(self,output,y):
        
        sampleLosses = self.check(output,y)
        
        #average loss 
        data_loss= np.mean(sampleLosses)
        
        #return the loss
        return data_loss

#Cross Entropy Loss  inheriting the loss class  
class CCE(loss):
    #for forward pass
    def check(self,y_pred,y_true):
        
        #number of samples
        x = len(y_pred)
        
        #clip data to prevent divison by 0
        y_pred_clipped = np.clip(y_pred ,1e-7,1 - 1e-7)
        
        
        #probablities for target values
        confidence = y_pred_clipped[range(x),y_true]
        negLikely = -np.log(confidence)
        
        #Return losses
        return negLikely
    
    #for back pass
    def backward(self,dval,y_true):
        
        #number of samples
        x=len(dval)
        
        # Number of labels in every sample
        # use the first sample to count them
        
        label=len(dval[0])
        
        if len(y_true.shape)==1:
            y_true=np.eye(label)[y_true] #
        
        # Calculate gradient
        self.dinputs =-y_true/dval
        # Normalize gradient
        self.dinputs =self.dinputs/x
       
    
class optimizer: #SGD
    
    #initialize optimizer 
    #set default settings
    def __init__(self , l_rate=1.,decay=0.,p=0.):
        self.l_rate = l_rate
        self.curr_l_rate=l_rate
        self.decay = decay
        self.iter =0
        self.p =p
    
    #Call before any parameters are updated
    #After every back pass 
    def initial_update_param(self):
        if self.decay:
            self.curr_l_rate = self.l_rate *(1./(1+self.decay*self.iter))
    
    #Update the parameters
    def update_params(self,layer):
        
        #if momentum is used
        if self.p:
            
            #if the layer doesnt have momentum atrribute create them filled with 0
            
            if not hasattr(layer,'wieght_momentums'):
                layer.weight_p = np.zeros_like(layer.weights)
                layer.bias_p = np.zeros_like(layer.biases)
            
            # Build weight updates with momentum - take previous
            # updates multiplied by retain factor and update with
            # current gradients
            
            w_updates= self.p*layer.weight_p - self_curr_l_rate*layer.dweights
            layer.weight_p = w_updates
            
            
            # Build bias updates
            b_updates= self.p*layer.bias_p - self_curr_l_rate*layer.dbiases
            layer.bias_p = b_updates
        
        # SGD updates (as before momentum update)
        else:
            w_updates =-self.curr_l_rate * layer.dweights
            b_updates =-self.curr_l_rate * layer.dbiases
            
        # Update weights and biases using either
        #  momentum updates
        
        layer.weights+=w_updates
        layer.biases+=b_updates
    
    #update iterations after every update
    def post_update_params(self):
        self.iter+=1

# Adam optimizer        
class  Adam_op:
    
    # Initialize optimizer - set settings
    def __init__(self , l_rate=.001,decay=0.,ep=1e-7,b1=0.9,b2=0.999):
        self.l_rate = l_rate
        self.curr_l_rate=l_rate
        self.decay = decay
        self.iter =0
        self.ep =ep
        self.b1=b1
        self.b2=b2
    
    #Call before any parameters are updated
    #After every back pass
    def initial_update_param(self):
        if self.decay:
            self.curr_l_rate = self.l_rate *(1./(1+self.decay*self.iter))
            
    #Update the parameters
    def update_params(self,layer):
        
        #If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer,'w_cache'):
            layer.w_p =np.zeros_like(layer.weights)
            layer.w_cache=np.zeros_like(layer.weights)
            layer.bias_p=np.zeros_like(layer.biases)
            layer.bias_cache=np.zeros_like(layer.biases)
            
        #Update momentum with current gradients
        layer.w_p = self.b1*layer.w_p + (1-self.b1)*layer.dweights
        layer.bias_p=self.b1*layer.bias_p + (1-self.b1)*layer.dbiases
        
        # Get corrected momentum
        # self.iteration is 0 at first pass
        # and we need to start with 1 here
        w_p_corrected = layer.w_p/(1-self.b1**(self.iter +1))
        bias_p_corrected = layer.bias_p/(1-self.b1**(self.iter +1))
        
        # Update cache with squared current gradients
        layer.w_cache = self.b2 * layer.w_cache + (1-self.b2)*layer.dweights**2
        layer.bias_cache = self.b2*layer.bias_cache+(1-self.b2)*layer.dbiases**2
        
        # Get corrected cache
        w_cache_corrected = layer.w_cache/(1-self.b2)**(self.iter+1)
        bias_cache_corrected = layer.bias_cache/(1-self.b2)**(self.iter+1)
        
        
        # SGD parameter update + normalization
        # with square rooted cache
        
        layer.weights+=-self.curr_l_rate*w_p_corrected/(np.sqrt(w_cache_corrected)+self.ep)
        layer.biases+=-self.curr_l_rate*bias_p_corrected/(np.sqrt(bias_cache_corrected)+self.ep)
    
    #update iterations after every update    
    def post_update_params(self):
        self.iter+=1
        

        
        
#Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step        
        
class Activation_softmax_LCCE():
    
    # Creates activation and loss function objects
    def __init__(self):
        self.activation=act_softmax()
        self.loss=CCE()
    
    # Forward pass
    def forward(self,inputs,y_true):
        
        # Output layer's activation function
        self.activation.fsoftmax(inputs)
        
        #store the output
        self.output=self.activation.output
        
        #Calculate and return loss value
        return self.loss.cal(self.output,y_true)
    
    # Backward pass    
    def backpass(self,dval,y_true):
        
        # Number of samples
        x=len(dval)
        if len(y_true.shape)==2:
            y_true = np.argmax(y_true,axis=1)
        
        # Copy so we can safely modify
        self.dinputs =dval.copy()
        
        # Calculate gradient
        self.dinputs[range(x),y_true]-=1
        
        # Normalize gradient
        self.dinputs= self.dinputs/x


In [4]:
input2=data.loc[:,['Age', 'Weight', 'Delivery phase', 'HB', 'IFA', 'BP', 'Residence']]
result=data['Result']

#Performing test train split
input1,x_test, result1, y_test = train_test_split(input2, data['Result'], test_size=0.35, random_state=0)

#creating first model
model1=NN(numHlayers=1)
model1.fit(input1,result1)

epoch:0 ,accuracy:0.742 ,loss:0.7447905083531015 ,lr:0.05,
epoch:100 ,accuracy:0.774 ,loss:0.5239543559468138 ,lr:0.04999752512250644,
epoch:200 ,accuracy:0.758 ,loss:0.5166351849101865 ,lr:0.04999502549496326,
epoch:300 ,accuracy:0.742 ,loss:0.5417555170206504 ,lr:0.049992526117345455,
epoch:400 ,accuracy:0.742 ,loss:0.5206480432822728 ,lr:0.04999002698961558,
epoch:500 ,accuracy:0.742 ,loss:0.5150306575816576 ,lr:0.049987528111736124,
epoch:600 ,accuracy:0.742 ,loss:0.520120660701345 ,lr:0.049985029483669646,
epoch:700 ,accuracy:0.742 ,loss:0.5182203358794077 ,lr:0.049982531105378675,
epoch:800 ,accuracy:0.742 ,loss:0.5173470114001129 ,lr:0.04998003297682575,
epoch:900 ,accuracy:0.742 ,loss:0.5170907191652694 ,lr:0.049977535097973466,
epoch:1000 ,accuracy:0.742 ,loss:0.5266011470534242 ,lr:0.049975037468784345,
epoch:1100 ,accuracy:0.742 ,loss:0.5290970950399508 ,lr:0.049972540089220974,
epoch:1200 ,accuracy:0.742 ,loss:0.521504948013835 ,lr:0.04997004295924593,
epoch:1300 ,accuracy:

epoch:10700 ,accuracy:0.774 ,loss:0.5142655439015246 ,lr:0.049733948243869425,
epoch:10800 ,accuracy:0.758 ,loss:0.49238080157328107 ,lr:0.04973147490127059,
epoch:10900 ,accuracy:0.758 ,loss:0.49032463958491157 ,lr:0.04972900180466547,
epoch:11000 ,accuracy:0.758 ,loss:0.49018864122029077 ,lr:0.049726528954017385,


In [5]:
#predict values using model 1
y=model1.predict(x_test,y_test)

accuracy:0.853 ,lr:0.04972650422675286,


In [6]:
model1.CM(y_test.tolist(),y)

Confusion Matrix : 
[[ 3  0]
 [ 5 26]]


Precision : 1.0
Recall : 0.8387096774193549
F1 SCORE : 0.9122807017543859


In [7]:
#model 2

In [8]:
model2=NN(numHlayers=1)
model2.fit(input1,result1)
#training model 2


epoch:0 ,accuracy:0.258 ,loss:2.778489961161019 ,lr:0.05,
epoch:100 ,accuracy:0.742 ,loss:0.5351005729928315 ,lr:0.04999752512250644,
epoch:200 ,accuracy:0.758 ,loss:0.542391976620342 ,lr:0.04999502549496326,
epoch:300 ,accuracy:0.742 ,loss:0.5243466758835301 ,lr:0.049992526117345455,
epoch:400 ,accuracy:0.742 ,loss:0.5291152792710466 ,lr:0.04999002698961558,
epoch:500 ,accuracy:0.742 ,loss:0.525159749676431 ,lr:0.049987528111736124,
epoch:600 ,accuracy:0.742 ,loss:0.5230158539423045 ,lr:0.049985029483669646,
epoch:700 ,accuracy:0.742 ,loss:0.5252456946354428 ,lr:0.049982531105378675,
epoch:800 ,accuracy:0.742 ,loss:0.5256125049094207 ,lr:0.04998003297682575,
epoch:900 ,accuracy:0.742 ,loss:0.5256951816421523 ,lr:0.049977535097973466,
epoch:1000 ,accuracy:0.742 ,loss:0.5218899764083991 ,lr:0.049975037468784345,
epoch:1100 ,accuracy:0.742 ,loss:0.5204481978822335 ,lr:0.049972540089220974,
epoch:1200 ,accuracy:0.758 ,loss:0.5242782000720241 ,lr:0.04997004295924593,
epoch:1300 ,accuracy:0

epoch:10900 ,accuracy:0.758 ,loss:0.4868556284980015 ,lr:0.04972900180466547,
epoch:11000 ,accuracy:0.758 ,loss:0.4721749890848062 ,lr:0.049726528954017385,


In [9]:
y=model2.predict(x_test,y_test)
#testing model 2

accuracy:0.765 ,lr:0.04972650422675286,


In [10]:
model2.CM(y_test.tolist(),y)

Confusion Matrix : 
[[ 0  0]
 [ 8 26]]


Precision : 1.0
Recall : 0.7647058823529411
F1 SCORE : 0.8666666666666666


In [11]:
#SMOTE Synthetic Minority Oversampling Technique like behaviour 

In [12]:
data2=data[(data['Result']==0)]
data2.head(3)
new_data=pd.concat([data,data2])
#new_data=pd.concat([new_data,data2])
new_data.head(3)
new_data=new_data.sample(frac=1)
new_data.head(3)
new_input=new_data.loc[:,['Age', 'Weight', 'Delivery phase', 'HB', 'IFA', 'BP', 'Residence']]
new_result=new_data['Result']
input1,x_test, result1, y_test = train_test_split(new_input, new_result, test_size=0.33, random_state=0)



In [13]:
#creating testing and training model 3
model3=NN(numHlayers=1)
model3.fit(input1,result1)

epoch:0 ,accuracy:0.550 ,loss:0.8850256014982048 ,lr:0.05,
epoch:100 ,accuracy:0.650 ,loss:0.6245100929138115 ,lr:0.04999752512250644,
epoch:200 ,accuracy:0.662 ,loss:0.6106375572067992 ,lr:0.04999502549496326,
epoch:300 ,accuracy:0.662 ,loss:0.6049313942556622 ,lr:0.049992526117345455,
epoch:400 ,accuracy:0.675 ,loss:0.5837776734387793 ,lr:0.04999002698961558,
epoch:500 ,accuracy:0.662 ,loss:0.6042811588169344 ,lr:0.049987528111736124,
epoch:600 ,accuracy:0.675 ,loss:0.5915195018224331 ,lr:0.049985029483669646,
epoch:700 ,accuracy:0.600 ,loss:0.5932878292741585 ,lr:0.049982531105378675,
epoch:800 ,accuracy:0.450 ,loss:0.640743395204362 ,lr:0.04998003297682575,
epoch:900 ,accuracy:0.450 ,loss:0.6387380737357474 ,lr:0.049977535097973466,
epoch:1000 ,accuracy:0.450 ,loss:0.6395730058639403 ,lr:0.049975037468784345,
epoch:1100 ,accuracy:0.450 ,loss:0.638149601361453 ,lr:0.049972540089220974,
epoch:1200 ,accuracy:0.450 ,loss:0.6374011405413754 ,lr:0.04997004295924593,
epoch:1300 ,accuracy:

epoch:10700 ,accuracy:0.450 ,loss:0.6194536815935554 ,lr:0.049733948243869425,
epoch:10800 ,accuracy:0.450 ,loss:0.6194176871222634 ,lr:0.04973147490127059,
epoch:10900 ,accuracy:0.450 ,loss:0.6194592831746141 ,lr:0.04972900180466547,
epoch:11000 ,accuracy:0.450 ,loss:0.6193483403912161 ,lr:0.049726528954017385,


In [14]:
y=model3.predict(x_test,y_test)

accuracy:0.300 ,lr:0.04972650422675286,


In [15]:
model3.CM(y_test.tolist(),y)

ZeroDivisionError: division by zero